# Resources

* https://python.langchain.com/docs/integrations/document_loaders/recursive_url

In [ ]:
# !pip install -r requirements.txt

# Imports

In [2]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Parsing Data

In [3]:
from bs4 import BeautifulSoup as Soup

url = "https://www.linkedin.com/legal/privacy-policy"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

In [4]:
docs[0].metadata['source']

'https://www.linkedin.com/legal/privacy-policy'

In [5]:
import tiktoken
tokenizer = tiktoken.get_encoding('cl100k_base')
# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = tiktoken_len
)

In [7]:
# !pip install -q -U faiss-cpu tiktoken sentence-transformers

In [8]:
documents = text_splitter.transform_documents(docs)

In [9]:
documents[0].metadata

{'source': 'https://www.linkedin.com/legal/privacy-policy',
 'title': 'LinkedIn Privacy Policy',
 'language': 'en'}

In [10]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore('./cache/')
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
core_embeddings_model = HuggingFaceEmbeddings(
    model_name = embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(documents, embedder)

In [ ]:
query = 'What is the data collection policy of linkedin?'
query_vector = core_embeddings_model.embed_query(query)
embedded_docs = vector_store.similarity_search_by_vector(query_vector, k = 4)

for doc in embedded_docs:
    print(doc.page_content)

In [12]:
# !pip install huggingface-hub -q

In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
import torch
import transformers
model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [15]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

In [16]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
)

In [17]:
# !pip install xformers

In [18]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [19]:
retriever = vector_store.as_retriever()

In [20]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

In [ ]:
qa_with_sources_chain({"query" : "What is the data collection policy of linkedin?"})